# Estudos de caso

Neste notebook vamos apresentar alguns exemplos de aplicações bem simples.

Não se esqueça de configurar o ambiente de execução para v2-8 TPU

In [40]:
!git clone https://github.com/Programacao-Paralela-e-Distribuida/MPI

Cloning into 'MPI'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 146 (delta 10), reused 5 (delta 5), pack-reused 120 (from 1)
Receiving objects: 100% (146/146), 3.94 MiB | 10.52 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [41]:
!mv MPI/* .
!rm -rf MPI
!ls

Makefile  README.md  bin  docs	jupiter  lncc  requirements.txt  src


#Método do trapézio

O valor de uma integral definida pode ser aproximado por vários métodos numéricos.  Um dos métodos mais utilizados é o método do trapézio, onde o valor da integral de uma função, que é definido como a área sob a curva da função até o eixo x, é aproximado pela soma da área de diversos trapézios, segundo a fórmula:

$
\int_a^b f(x) \, dx \approx h \times \left[ \frac{f(x_0)}{2} + \sum_{i=1}^{n-1} f(x_i) + \frac{f(x_n)}{2} \right],
$

onde:

- $a$ e $b$ são os limites de integração.
- $n$ é o número de subintervalos.
- $h = \frac{b-a}{n}$ é a largura de cada subintervalo.
- $x_0 = a$, $x_n = b$, e $x_i = a + i \cdot h$ para $i = 1, 2, \dots, n-1$.

No exemplo as seguir, vamos assumir $f(x) = exp (x)$. Os limites utilizados serão $a = 0$ e $b = 1$, então $exp(0) = 1$ e $exp(1)= 2,718282$, então o valor aproximado desta integral é $1,718282$.

Na paralelização deste algoritmo, uma melhor forma de distribuição é atribuir o cálculo de $f(x_i)$ alternadamente a cada processo. Deste modo temos:


In [42]:
!cat src/mpi_trapezio.c


#include <stdio.h>
#include <math.h>
#include "mpi.h"

double f(double x) {
        double return_val;
        return_val = exp(x); /* Função exponencial */
        return return_val;
}

int main(int argc, char *argv[]) { /* mpi_trapezio.c  */
int meu_ranque, num_procs;  /* respectivamente q e p */
double a = 0.0,  b = 1.0;   /* Limites da integral */
double tempo_inicial, tempo_final; /* Tempo de execução */
long int n = 100000000;     /* Número de trapezoides */
double x, h;                /* x e h, a base do trapezoide */
double integral=0.0, total; /* Integral de cada processo e total */
int origem, destino = 0;    /* Origem e destino das mensagens */
int etiq = 3;               /* Uma etiqueta qualquer */

    /* Inicia o MPI e determina o ranque e o número de processos ativos  */
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &meu_ranque);
    MPI_Comm_size(MPI_COMM_WORLD, &num_procs);
    /* h é o mesmo para todos os processos */
    h = (b - a)/n;   
    /* O pro

Aqui você pode subsituir N por um número que dê um tempo de execução de pelo menos alguns segundos. Isso depende da máquina que estiver executando. Se necesssário adicione a opção *--allow-run-as-root* ao comando *mpirun*. O número máximo de núcleos que você vai utilizar depende também da configuração da máquina que você estiver utilizando.

Elabore um gráfico de speed-up e eficiência com os resultados obtidos.

In [46]:

!mpicc src/mpi_trapezio.c -o bin/mpi_trapezio -lm
!mpirun -np 2 ./bin/mpi_trapezio
!mpirun -np 4 ./bin/mpi_trapezio
!mpirun -np 8 ./bin/mpi_trapezio
!mpirun -np 10 ./bin/mpi_trapezio
!mpirun -np 12 ./bin/mpi_trapezio
!mpirun -np 14 ./bin/mpi_trapezio
!mpirun -np 16 ./bin/mpi_trapezio

Foram gastos 23.6 segundos
Com n = 10000000000 trapezoides, a estimativa
da integral de 0.000000 até 1.000000 = 1.718282 
Foram gastos 14.0 segundos
Com n = 10000000000 trapezoides, a estimativa
da integral de 0.000000 até 1.000000 = 1.718282 
Foram gastos 8.9 segundos
Com n = 10000000000 trapezoides, a estimativa
da integral de 0.000000 até 1.000000 = 1.718282 
Foram gastos 8.7 segundos
Com n = 10000000000 trapezoides, a estimativa
da integral de 0.000000 até 1.000000 = 1.718282 
Foram gastos 10.0 segundos
Com n = 10000000000 trapezoides, a estimativa
da integral de 0.000000 até 1.000000 = 1.718282 
Foram gastos 15.5 segundos
Com n = 10000000000 trapezoides, a estimativa
da integral de 0.000000 até 1.000000 = 1.718282 
Foram gastos 10.3 segundos
Com n = 10000000000 trapezoides, a estimativa
da integral de 0.000000 até 1.000000 = 1.718282 


Lembre-se também que o MPI oferece uma operação coletiva de redução, que pode ser utilizada para simplificar o código e otimizar a execução em paralelo.
Assim, podemos reescrever as últimas linhas do programa substituindo o laço de envio dos resultados parciais pela rotina de redução, que é uma forma muito mais eficiente para o cálculo do valor final da integral. Copie e altere o código acima para esta nova versão. Compile e execute o código comparando com a versão anterior.

In [ ]:
%%writefile src/mpi_trapezio_novo.c
#include <stdio.h>
#include <math.h>
#include "mpi.h"

double f(double x) {
        double return_val;
        return_val = exp(x); /* Função exponencial */
        return return_val;
}

int main(int argc, char *argv[]) { /* mpi_trapezio.c  */
int meu_ranque, num_procs;  /* respectivamente q e p */
double a = 0.0,  b = 1.0;   /* Limites da integral */
double tempo_inicial, tempo_final; /* Tempo de execução */
long int n = 1000000000;     /* Número de trapezoides */
double x, h;                /* x e h, a base do trapezoide */
double integral=0.0, total; /* Integral de cada processo e total */
int origem, destino = 0;    /* Origem e destino das mensagens */
int etiq = 3;               /* Uma etiqueta qualquer */

    /* Inicia o MPI e determina o ranque e o número de processos ativos  */
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &meu_ranque);
    MPI_Comm_size(MPI_COMM_WORLD, &num_procs);
    /* h é o mesmo para todos os processos */
    h = (b - a)/n;
    /* O processo 0 calcula o valor de f(x)/2 em a e b */
    if (meu_ranque == 0) {
        tempo_inicial = MPI_Wtime();
        integral = (f(a) + f(b))/2.0;
    }
    /* Cada processo calcula a integral aprox. sobre n/num_procs trapézios */
    for (x = a+h*(meu_ranque+1); x < b ; x += num_procs*h) {
         integral += f(x);
    }
    integral = integral*h;
    /* O processo 0 soma as integrais parciais recebidas */
    if (meu_ranque == 0) {
        total = integral;
        for (origem = 1; origem < num_procs; origem++) {
             MPI_Recv(&integral, 1, MPI_DOUBLE, origem, etiq, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
             total += integral;
        }
    }
     /* Os demais processos enviam as integrais parciais para o processo 0 */
    else {
        MPI_Send(&integral, 1, MPI_DOUBLE, destino, etiq, MPI_COMM_WORLD);
    }
    /* Imprime o resultado */
    if (meu_ranque == 0) {
        tempo_final = MPI_Wtime();
        printf("Foram gastos %3.1f segundos\n",tempo_final-tempo_inicial);
        printf("Com n = %ld trapezoides, a estimativa\n", n);
        printf("da integral de %lf até %lf = %lf \n", a, b, total);
    }
    MPI_Finalize();
    return(0);
}


Writing src/mpi_trapezio_novo.c


In [50]:
!mpicc src/mpi_trapezio_novo.c -o bin/mpi_trapezio_novo -lm
!mpirun -allow-run-as-root -np 4 ./bin/mpi_trapezio_novo

Foram gastos 12.8 segundos
Com n = 10000000000 trapezoides, a estimativa
da integral de 0.000000 até 1.000000 = 1.718282 


#Números primos

O programa em questão serve para determinar a quantidade de números primos entre 0 e um determinado valor inteiro N.
Embora possa parecer um programa trivial a princípio, ele tem algumas particularidades que o tornam um problema interessante.
Na matemática, o Teorema do Número Primo (TNP) descreve a distribuição assintótica dos números primos entre os inteiros positivos.
Ele formaliza a ideia intuitiva de que os números primos tornam-se menos comuns à medida que N aumenta, quantificando precisamente a taxa em que isso ocorre.
<br><br>
O que isso implica na programação paralela?
Bom, a nossa primeira tentativa de paralelização seria dividir o total de N números igualmente entre os P processadores disponíveis, ou seja, N/P valores para cada uma das threads.
No entanto, há uma implicação importante: a distribuição dos números primos não é uniforme entre os inteiros.
Isso torna essa abordagem de divisão direta ineficaz, pois as threads que receberem intervalos com uma maior concentração de números primos terão uma carga de trabalho significativamente maior.
Isso pode levar a um desequilíbrio no desempenho, com algumas threads concluindo suas tarefas mais rapidamente, enquanto outras permanecem ocupadas, resultando em subutilização dos recursos disponíveis.
<br><br>
A solução apresentada descarta todos os números pares de início, pois obviamente eles não são primos.
Em seguida, é verificado se $N$ é divisível por algum número ímpar entre $0$ e a raiz quadrada de $N$. Por que isso?
Se um número composto $N$ pode ser fatorado como $N = a * b$, então pelo menos um dos fatores ($a$ ou $b$) deve ser menor ou igual à raiz quadrada de $N$.
Se ambos $a$ e $b$ forem maiores que a raiz quadrada de $N$, então ao multiplicá-los, o resultado seria maior que $N$. Perfeito?
<br><br>
Uma estratégia mais simples de paralelização seria distribuir as tarefas entre os processos em lote, passando uma faixa contínua de valores para cada processo verificar a quantidade de números primos no intervalo recebido.
Mas como já vimos, essa não é uma boa abordagem, pois a distribuição de números primos não é uniforme ao longo do espaço de números inteiros.
Um alternativa possível é cada processo verificar alternadamente se cada número ímpar é primo, o que na prática significa que cada processo comece verificando um número ímpar diferente e saltando $num\_proc$ números ímpares entre cada número ímpar verificado.
<br><br>

In [ ]:
!cat src/mpi_primos.c

#include <stdio.h>
#include <stdlib.h>
#include "mpi.h"
#include <math.h>

int primo (long int n) { /* mpi_primos.c  */
	int i;
       
	for (i = 3; i < (int)(sqrt(n) + 1); i+=2) {
			if(n%i == 0) return 0;
	}
	return 1;
}

int main(int argc, char *argv[]) {
	double t_inicial, t_final;
	int cont = 0, total = 0;
	long int i, n;
	int meu_ranque, num_procs, inicio, salto;

	if (argc < 2) {
        	printf("Valor inválido! Entre com um valor do maior inteiro\n");
       	 	return 0;
    	} else {
        	n = strtol(argv[1], (char **) NULL, 10);
       	}

	MPI_Init(&argc, &argv);
	MPI_Comm_rank(MPI_COMM_WORLD, &meu_ranque);
	MPI_Comm_size(MPI_COMM_WORLD, &num_procs);	
    t_inicial = MPI_Wtime();
    inicio = 3 + meu_ranque*2;
    salto = num_procs*2;
	for (i = inicio; i <= n; i += salto) 
	{	
		if(primo(i) == 1) cont++;
	}
		
	if(num_procs > 1) {
		MPI_Reduce(&cont, &total, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);
	} else {
		total = cont;
	}
	
	t_final = MPI_Wtime();

	if (meu_ranque ==

In [ ]:
!mpicc src/mpi_primos.c -o bin/mpi_primos -lm
!mpirun --allow-run-as-root  -np 1 ./bin/mpi_primos 10000000

Quant. de primos entre 1 e n: 664579 
Tempo de execucao: 12.209 


#Multiplicação matriz-vetor

Um outro estudo de caso que faremos é a multiplicação de uma matriz por um vetor, que tem como resultado um vetor.

Note que cada elemento do vetor resultado é o produto escalar de uma linha da matriz com o vetor de entrada.

Há várias maneiras de decompor o problema e distribuir os dados entre os diversos processos. Cada uma dessas formas de decomposição tem as suas vantagens e desvantagens, além de complexidades distintas.
Por simplicidade, vamos assumir que utilizaremos a primeira alternativa de distribuição de dados, com cada processo possuindo um bloco de linhas da matriz A e os vetores b e c replicados em cada processo.

In [ ]:
!cat src/mpi_mxv.c

#include <stdio.h>
#include <stdlib.h>
#include "mpi.h" 

void mxv(int n, double* A, double* b, double* c);

int main(int argc, char *argv[]) { /* mpi_mxv.c  */
double *A,*Aloc, *b,*c;
/* matriz m x n por um vetor de dimensão n */
int i, j, m, n;                     
int meu_ranque, num_procs, raiz=0;
double start, finish, loc_elapsed, elapsed;

   MPI_Init(&argc, &argv);
   MPI_Comm_rank(MPI_COMM_WORLD, &meu_ranque);
   MPI_Comm_size(MPI_COMM_WORLD, &num_procs);
   
   if (meu_ranque == 0) {
      printf("Por favor entre com n: \n");
      scanf("%d",&n);
      printf("\n");
    }
   
    m = num_procs;

    MPI_Bcast(&n, 1, MPI_INT, raiz, MPI_COMM_WORLD);

    if (meu_ranque == 0) {
        printf("Valor de m: %d e  n: %d \n", m,n);
        A=(double*) malloc(m*n*sizeof(double));
        Aloc=(double *)malloc(m*n*sizeof(double));
        b=(double*) malloc(n*sizeof(double));
        c=(double*) malloc(m*sizeof(double));
     } else {
        Aloc=(double *) malloc(n*sizeof(double));


In [ ]:
!mpicc src/mpi_mxv.c -o bin/mpi_mxv -lm
!echo 10000 | mpirun --allow-run-as-root  -np 1 ./bin/mpi_mxv

Por favor entre com n: 

Valor de m: 1 e  n: 10000 
Atribuindo valor inicial à matriz A e ao vetor b
Tempo total = 4.720500e-05
